<a href="https://colab.research.google.com/github/vishaljoshi24/DnD-AutoPrompt/blob/main/classifying_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 28.1 MB/s eta 0:00:00


SET-UP

In [ ]:
import dspy

lm = dspy.LM("ollama_chat/llama3.2:1b", api_base = "http://localhost:11434", api_key="b229890ea0664f6193770b4b470f3e74.nJRqYf9TBF9MX24r4fUQQqqB")
dspy.configure(lm=lm)

DATA

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
dataset = load_dataset("garrykuwanto/crd3_training_pairs", download_mode="force_redownload")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


crd3_training_pairs_no_llm.json:   0%|          | 0.00/773M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2360582 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
query = []
passage = []
for i in range(len(df.train[0:])):
  query.append(df.train[i]['query'])
  passage.append(df.train[i]['passage'])

df1 = pd.DataFrame({'context': query, 'current turn': passage})

In [ ]:
df1

,context,current turn
0,Dialogue context: MATT: And welcome to tonight...,ZAC: Testicular.
1,Dialogue context: ZAC: Fantastical? MATT: Fant...,MATT: It is. It is a testicle episode of Criti...
2,Dialogue context: MATT: Fantastical. ZAC: Test...,ZAC: Cue cut to Ashley!
3,Dialogue context: ZAC: Testicular. MATT: It is...,MATT: Yay! Hi Ashley! She's dancing for it. So...
4,Dialogue context: MATT: It is. It is a testicl...,ZAC: We've got a couple of quick announcements...
...,...,...
2360577,Player says: Not continuing the song... (all l...,"DM responds: Guys, thank you so much for watch..."
2360578,Player says: Not Taliesin.,"DM responds: Not Taliesin, sorry. Liam, Marish..."
2360579,Player says: Saturday.,DM responds: Sorry. Saturday. Friday. It's -
2360580,Player says: It technically is tomorrow.,"DM responds: No, it's not tomorrow yet. Sorry...."


In [ ]:
df2 = df1[500:510]

df2

,context,current turn
500,"Dialogue context: MATT: Laura, can you roll a ...",LAURA: Just regular?
501,Dialogue context: LAURA: (nervous laugh) Yeah-...,MATT: Just a regular constitution save.
502,Dialogue context: TRAVIS: 17! LAURA: Just regu...,LAURA: Seven.
503,Dialogue context: LAURA: Just regular? MATT: J...,MATT: Seven? Okay. You suffer six points of po...
504,Dialogue context: MATT: Just a regular constit...,"SAM: Grog, you suffer no poison damage. We had..."
505,Dialogue context: LAURA: Seven. MATT: Seven? O...,TRAVIS: Heroes' Feast!
506,Dialogue context: MATT: Seven? Okay. You suffe...,"LAURA: And Trinket ate it, too!"
507,"Dialogue context: SAM: Grog, you suffer no poi...",MARISHA: That's true!
508,Dialogue context: TRAVIS: Heroes' Feast! LAURA...,LAURA: We're immune to poison?
509,"Dialogue context: LAURA: And Trinket ate it, t...",MARISHA: We're immune to fear and poison.


In [ ]:
df2.insert(2, 'quality', ['goal-oriented', 'DM turn', 'goal-oriented', 'DM turn', 'collaborative action', 'open ended', 'contextually relevant', 'collaborative', 'goal-oriented', 'open ended'  ])

In [ ]:
examples = []

for context, turn, quality in df2.values:
  examples.append(dspy.Example(context=context, turn=turn, quality=quality).with_inputs("context", "current turn", "quality"))

examples

[Example({'context': 'Dialogue context: MATT: Laura, can you roll a constitution save for Trinket? LAURA: (nervous laugh) Yeah-- TRAVIS: 17!', 'turn': 'LAURA: Just regular?', 'quality': 'goal-oriented'}) (input_keys={'quality', 'context', 'current turn'}),
 Example({'context': 'Dialogue context: LAURA: (nervous laugh) Yeah-- TRAVIS: 17! LAURA: Just regular?', 'turn': 'MATT: Just a regular constitution save.', 'quality': 'DM turn'}) (input_keys={'quality', 'context', 'current turn'}),
 Example({'context': 'Dialogue context: TRAVIS: 17! LAURA: Just regular? MATT: Just a regular constitution save.', 'turn': 'LAURA: Seven.', 'quality': 'goal-oriented'}) (input_keys={'quality', 'context', 'current turn'}),
 Example({'context': 'Dialogue context: LAURA: Just regular? MATT: Just a regular constitution save. LAURA: Seven.', 'turn': 'MATT: Seven? Okay. You suffer six points of poison damage, Trinket suffers 12 points of poison damage.', 'quality': 'DM turn'}) (input_keys={'quality', 'context', 

SIGNATURES

In [ ]:
from typing import Literal


from typing import Literal

class TurnClassifier(dspy.Signature):
    data: dict[str, str] = dspy.InputField(desc = """
  {'context': 'The three previous game turns which describe a player's action or their dialogue.',
   'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
   }""")
    behaviour_class: Literal['collaborative',
                             'contextually-relevant',
                             'goal-oriented',
                             'open-ended'] = dspy.OutputField(desc="The feature of a player's turn within a Dungeons & Dragons game: collaborative, open-ended, goal-oriented, contextually relevant")

classify = dspy.ChainOfThought(BehaviourClassifier)

# class PlayerInstruction(dspy.Signature):
#   "Generate a prompt which instructs an agent how to behave as a D&D player based on the labelled quality for a specific gameplay turn"

#   examples: dict[str, str] = dspy.InputField(desc = """
#   {'context': 'The three previous game turns which describe a player's action or their dialogue.',
#    'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
#    'behaviour': 'The behavious exhibited by a player in their turn within a Dungeons & Dragons game: open ended, goal oriented, contextually relevant and collaborative. DM turns should be ignored.'
#   }
#   """)
#   instruction: str = dspy.OutputField(desc = "instruction on how to behave within a D&D game which describes the quality label of a particular turn.")

  # context: dict[str, str] = dspy.InputField(desc = "The three previous game turns which describe a player's action or their dialogue")
  # turn: dict[str,str] = dspy.InputField(desc = "The current turn taken by a player, which can include a description of an action or a piece of dialogue.")
  # quality: dict[str,str] = dspy.InputField(desc="The quality of a player's actions within a Dungeons & Dragons game: open-ended, goal-oriented, contextually relevant, collaborative.")




MODULES

In [ ]:
# class PromptGenerator(dspy.Module):
#   def __init__(self):
#     # self.classifier = dspy.ChainOfThought(Quality, caching = False)
#     self.generator = dspy.ChainOfThought(PlayerInstruction, caching = False)

#   def forward(self, examples, **kwargs):
#     # quality_class = self.classifier(action=action).quality
#     prompt = self.generator(examples=examples).instruction
#     return prompt

In [ ]:
# generate_prompts = PromptGenerator()
# prompt_list = []
# for i in range(len(examples)):
#   prompt_list.append(generate_prompts(i))
# prompt_list

["{instruction=\nTo behave correctly according to the labelled quality of the current turn:\n- For 'open ended', be flexible and adapt to changing circumstances.\n- For 'goal oriented', focus on achieving a specific objective or outcome.\n- For 'contextually relevant', consider the context in which you are playing the game.\n- For 'collaborative', work with other players to achieve a common goal.\n\nExample: If it's your turn and you have a goal-oriented behaviour, make sure to clearly state what you want to achieve. Consider the context of the situation and adjust your approach accordingly.\n}\n\n[[ ## instruction completed ## ]]\n{instruction=}\n```",
 "If you're an open-ended player, use your initiative to take a bold step forward. You might:\n- Take an action that challenges the current status quo\n- Attempt to change the direction of the game (if possible)\n- Use your character's special abilities to influence the outcome\n\nIf you're a goal-oriented player, identify a new objecti

EVALUATION QUESTIONS



  clear = "Is the assessed prompt specific, unambiguous and easy to interpret?"

  relevant = "Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?"

  correct = "Is the assessed prompt correct with respect to the player quality that it is describing?"

  complete = "Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?"



EVALUATION SIGNATURES

In [ ]:
# class Assess(dspy.Signature):
#   """Assess the quality of a prompt along a specified dimension."""

#   assessed_text: list[str] = dspy.InputField()
#   assessment_question: str = dspy.InputField()
#   assessment_answer: bool = dspy.OutputField()

In [ ]:
# clarity_evaluation = dspy.ChainOfThought(Assess, caching=False)
# relevance_evaluation = dspy.ChainOfThought(Assess, caching=False)
# correctness_evaluation = dspy.ChainOfThought(Assess, caching=False)
# completeness_evaluation = dspy.ChainOfThought(Assess, caching=False)

In [ ]:
# clarity_list = []
# for i in prompt_list:
#   prompt_clarity = clarity_evaluation(assessed_text=i, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?")
#   clarity_list.append(prompt_clarity)

# clarity_list

# # clarity_evaluation(assessed_text=clarity_list, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?")

[Prediction(
     reasoning='The assessed prompt is not specific, open-ended, or contextually relevant. It does not provide clear instructions on how to behave correctly.',
     assessment_answer=False
 ),
 Prediction(
     reasoning='The prompted text appears to be somewhat open-ended, leaving room for interpretation. It may not be entirely clear what is being asked or expected.',
     assessment_answer=False
 ),
 Prediction(
     reasoning='The prompt is not specific, as it does not provide clear information about what constitutes good gameplay or character actions.',
     assessment_answer=False
 ),
 Prediction(
     reasoning='The assessed prompt is clear and concise.',
     assessment_answer=True
 ),
 Prediction(
     reasoning="The prompt is not goal-oriented because it does not specifically explore the game's narrative, develop character storylines, or engage with other players' contributions.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="The assessed prompt is

In [ ]:
# relevance_list = []
# for i in prompt_list:
#   prompt_relevance = relevance_evaluation(assessed_text=i, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?")
#   relevance_list.append(prompt_relevance)

# relevance_list

# # relevance_evaluation(assessed_text=example_1, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?")

[Prediction(
     reasoning='The prompt provided does not align with the categorised quality of D&D player behaviour that it is trying to express. The instruction section explicitly states criteria for each type of response, and this prompt fails to address all these requirements.',
     assessment_answer=False
 ),
 Prediction(
     reasoning="The prompt encourages players to take bold and active roles, suggesting an 'open-ended' approach. However, it also implies a level of planning and goal-oriented behavior from the player.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="When receiving a label for your gameplay turn, you should:\n\n1. Take note of the specific labelled quality rating.\n2. Reflect on what this means for your character's or their group's actions in that situation.\n3. Consider how to adjust your strategy or make informed decisions to improve your chances of success.",
     assessment_answer=False
 ),
 Prediction(
     reasoning='The prompt does not ali

In [ ]:
# correctness_list = []
# for i in prompt_list:
#   prompt_correctness = correctness_evaluation(assessed_text=i, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?")
#   correctness_list.append(prompt_correctness)

# correctness_list

# # correctness_evaluation(assessed_text=example_1, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?")

[Prediction(
     reasoning="The prompt appears to be assessing the player's ability to behave correctly according to a specified dimension. However, it does not provide clear guidance on which dimension or aspect of play it is evaluating. Therefore, I will assess the quality of this prompt as 'inadequate' for accurately describing a specified dimension.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="The assessed prompt correctly describes an open-ended player who is taking bold steps forward, which aligns with the quality characteristic of being 'open-minded'. The assessment question itself does not provide additional context or information to further clarify this aspect.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="When receiving a label for your gameplay turn, you should reflect on what this means for your character's or their group's actions in that situation to adjust strategy and make informed decisions.",
     assessment_answer=False
 ),
 Pred

In [ ]:
# completeness_list = []
# for i in prompt_list:
#   prompt_completeness = completeness_evaluation(assessed_text=i, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?")
#   completeness_list.append(prompt_completeness)

# completeness_list

# # completeness_evaluation(assessed_text=example_1, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?")

[Prediction(
     reasoning="The prompt does not fully express the quality of the player's behaviour, as it lacks specificity and clarity regarding secondary queries or implied context.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="The prompt does not fully express the quality of the player's behavior. It focuses on open-ended action and goal-oriented planning, but lacks clear instructions or guidance for taking bold steps forward or achieving long-term objectives.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="The prompt provides specific labelled quality ratings but lacks clear guidance on how these ratings relate to character actions and decisions. This suggests that the prompt may not fully capture the nuances of a player's behavior.",
     assessment_answer=False
 ),
 Prediction(
     reasoning="The prompt does not fully express the quality of the player's behaviour, as it lacks specificity about the context and secondary queries that might have 

In [ ]:
# class ScorePrompt(dspy.Module):
#   def __init__(self):
#     self.assessment = dspy.ChainOfThought(Assess, caching=False)

#   def forward(self, prompt_list, **kwargs):
#     score_list = []
#     for i in prompt_list:
#       assess_clarity = self.assessment(assessed_text=i, assessment_question="Is the assessed prompt specific, unambiguous and easy to interpret?").assessment_answer
#       assess_relevance = self.assessment(assessed_text=i, assessment_question="Does the prompt align with the categorised quality of D&D player behaviour that it is trying to express?").assessment_answer
#       assess_correctness = self.assessment(assessed_text=i, assessment_question="Is the assessed prompt correct with respect to the player quality that it is describing?").assessment_answer
#       assess_completeness = self.assessment(assessed_text=i, assessment_question="Does the prompt fully express the quality of the player's behaviour, taking into account additional context or secondary queries that might have been implied or included in the prompt?").assessment_answer

#       if assess_clarity == True:
#         assess_clarity = 1
#       else:
#         assess_clarity = 0
#       if assess_relevance == True:
#         assess_relevance = 1
#       else:
#         assess_relevance = 0
#       if assess_correctness == True:
#         assess_correctness = 1
#       else:
#         assess_correctness = 0
#       if assess_completeness == True:
#         assess_completeness = 1
#       else:
#         assess_completeness = 0

#       score = (assess_clarity + assess_relevance + assess_correctness + assess_completeness)/4
#       score_list.append(score)

#     return score_list

In [ ]:
# prompt_score_list = ScorePrompt()
# prompt_score_list(prompt_list=prompt_list)

[0.0, 0.0, 0.0, 0.5, 0.25, 0.75, 0.25, 0.5, 0.0, 0.25]